In [1]:
import numpy as np, xarray as xr, matplotlib.pyplot as plt, pandas as pd, s3fs

In [2]:
s3 = s3fs.S3FileSystem(key="K1CQ7M1DMTLUFK182APD", secret="3JuZAQm5I03jtpijCpHOdkAsJDNLNfZxBpM15Pi0", client_kwargs=dict(endpoint_url="https://rgw.met.no"))

In [3]:
base_path = 's3://escience2022/Ada/'
s3file = base_path + 'monthly/chlos_Omon_CESM2_esm-ssp585_r1i1p1f1_gn_201501-206412.nc'
file = s3.open(s3file)
ds = xr.open_dataset(file)
ds

/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/conventions.py:521: SerializationWarning: variable 'chlos' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


<xarray.Dataset>
Dimensions:    (time: 600, nlat: 384, nlon: 320, d2: 2, vertices: 4)
Coordinates:
    lat        (nlat, nlon) float64 ...
    lon        (nlat, nlon) float64 ...
  * nlat       (nlat) int32 1 2 3 4 5 6 7 8 ... 377 378 379 380 381 382 383 384
  * nlon       (nlon) int32 1 2 3 4 5 6 7 8 ... 313 314 315 316 317 318 319 320
  * time       (time) object 2015-01-15 13:00:00.000007 ... 2064-12-15 12:00:00
Dimensions without coordinates: d2, vertices
Data variables:
    chlos      (time, nlat, nlon) float32 ...
    time_bnds  (time, d2) object ...
    lat_bnds   (nlat, nlon, vertices) float32 ...
    lon_bnds   (nlat, nlon, vertices) float32 ...
Attributes: (12/45)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            C4MIP
    branch_method:          standard
    branch_time_in_child:   735110.0
    branch_time_in_parent:  735110.0
    case_id:                1525
    ...                     ...
    sub_experiment_id:      none
    table_id:               Omon
    tracking_id:            hdl:21.14100/4f91d14c-027d-47e7-a6d7-65f374e152af
    variable_id:            chlos
    variant_info:           CMIP6 esm-ssp585 experiments (2015-2100) with CAM...
    variant_label:          r1i1p1f1

In [10]:
def get_data_from_bucket():
    s3 = s3fs.S3FileSystem(key="K1CQ7M1DMTLUFK182APD", secret="3JuZAQm5I03jtpijCpHOdkAsJDNLNfZxBpM15Pi0",
                           client_kwargs=dict(endpoint_url="https://rgw.met.no"))
    base_path = 's3://escience2022/Ada/'
    full_path = base_path + 'monthly/chlos_Omon_CESM2_esm-ssp585_r1i1p1f1_gn_*.nc'
    remote_files = s3.glob(full_path)

    print(remote_files)
    fileset = [s3.open(file) for file in remote_files]
    return xr.open_mfdataset(fileset, combine='by_coords')
    
chlos = get_data_from_bucket()

['escience2022/Ada/monthly/chlos_Omon_CESM2_esm-ssp585_r1i1p1f1_gn_201501-206412.nc', 'escience2022/Ada/monthly/chlos_Omon_CESM2_esm-ssp585_r1i1p1f1_gn_206501-210012.nc']


/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/conventions.py:521: SerializationWarning: variable 'chlos' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


In [20]:
chlos.plot(x='lat', y='lon')

ValueError: Dataset.plot cannot be called directly. Use an explicit plot method, e.g. ds.plot.scatter(...)